# Change detection of chaotic time-series

In this note book, we apply our methods to a chaotic time-series generated by the Lorenz equation.

In [ ]:
import gudhi as gd
from gudhi.representations import Landscape
import numpy as np
import matplotlib.pyplot as plt
from functools import partial

In [ ]:
from mdl.model import Norm1D
from mdl.smdl import SMDL
from bocpd.mybocpd import BOCD, StudentT, constant_hazard
from mdl.ppm import get_K_mu_sigma
from mdl.wkc import get_WKC
from utils.evaluation import calc_auc_average, calc_falarms_benefit, InvRunLen
from utils.embedding import TimeDelayEmbedding

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Generate dataset
We generate a time-series using the Lorenz equation.

In [ ]:
def lorenz(x, y, z, p,r,b):
    x_dt = p*(y-x)
    y_dt = x*(r-z)-y
    z_dt = x*y-b*z
    return x_dt, y_dt, z_dt

def get_chaos_data(r,t):
    dt = 0.02
    num_steps = 1500
    p = 10
    b = 8/3
    x = [0.01]
    y = [0.01]
    z = [0.01]
    for i in range(num_steps):
        if i==(num_steps-2*t):
            r = 35
        if i==(num_steps-t):
            r = 40

        x_dt, y_dt, z_dt = lorenz(x[-1], y[-1], z[-1],p,r,b)
        x.append(x[-1] + (x_dt * dt))
        y.append(y[-1] + (y_dt * dt))
        z.append(z[-1] + (z_dt * dt))

    return x,y,z

## Time-delay embedding
Apply time-delay embedding to the time-series data and convert it to a series of three-dimensional point clouds. 

In [ ]:
data = get_chaos_data(28,500)[0]
TimeDelayWindow = TimeDelayEmbedding(250,1,1)
ex_data = TimeDelayWindow(data,0)
TimeDelay = TimeDelayEmbedding(3,5,1)
use_data = TimeDelay(ex_data,1)

In [ ]:
plt.plot(data)

## Number of optimal components in Persistence Parametric Model
We apply the PPM method to the PDs of the point clouds.

In [ ]:
Ks = []
max_K = 7
b = 1
for i in range(len(use_data)):
    ob_data = use_data[i]
    rips_complex = gd.RipsComplex(points=ob_data)
    simplex_tree = rips_complex.create_simplex_tree(max_dimension=2)
    diag = simplex_tree.persistence()
    A = simplex_tree.persistence_intervals_in_dimension(1)
    K, mu, sigma = get_K_mu_sigma(A, max_K, b)
    Ks.append(K)

In [ ]:
plt.plot(Ks)

We plot the centers of PPM at several time.

In [ ]:
for i in [100,600,1100]:
    ob_data = use_data[i]
    rips_complex = gd.RipsComplex(points=ob_data)
    simplex_tree = rips_complex.create_simplex_tree(max_dimension=2)
    diag = simplex_tree.persistence()
    A = simplex_tree.persistence_intervals_in_dimension(1)
    K, mu, sigma = get_K_mu_sigma(A, max_K, b)
    
    plt.scatter(A.T[0],A.T[1]-A.T[0])
    for l in range(K):
        plt.scatter(mu[l][0],mu[l][1],color="red")
    plt.show()

We smooth the series of the number of mixture components and apply Bayesian online change point detection (BOCPD).

In [ ]:
smooth = 3
smooth_Ks = [0]*(smooth-1)
for i in range(smooth-1,len(Ks)):
    smooth_Ks.append(np.mean(Ks[i-smooth+1:i+1]))

In [ ]:
ALPHA = 0.1
BETA = 1.0
KAPPA = 1.0
MU = 0.0
DELAY = 15
N_trial = 1
T = 200

for LAMBDA in [50,100,150,200,250,300]:
    for THRESHOLD in [0.1, 0.3]:
        scores_bocpd = []
        for i in range(N_trial):
            X = smooth_Ks

            # BOCPD
            bocd = BOCD(partial(constant_hazard, LAMBDA),
                        StudentT(ALPHA, BETA, KAPPA, MU), X)
            change_points = []
            scores = [np.nan] * DELAY
            for x in X[:DELAY]:
                bocd.update(x)
            for x in X[DELAY:]:
                bocd.update(x)
                if bocd.growth_probs[DELAY] >= THRESHOLD:
                    change_points.append(bocd.t - DELAY + 1)
                score = np.sum(bocd.growth_probs[:bocd.t - DELAY] * 1.0 / (1.0 + np.arange(1, bocd.t - DELAY + 1)))
                scores.append(score)

            scores_bocpd.append(scores)

        scores_bocpd = np.array(scores_bocpd)
        auc_list = calc_auc_average(scores_bocpd,np.array([250,750]), T=T)
        print('LAMBDA =', LAMBDA, 'THRESHOLD =', THRESHOLD, 'AUC:', np.mean(auc_list))

## Kernel Complexity of Persistence Non-Parametric Model
We apply the PNPM method to the PDs of the point clouds.

In [ ]:
KCs_PNPM = []
epsilon = 0.1
gamma = 0.7
param = 1.0
for i in range(len(use_data)):
    ob_data = use_data[i]
    rips_complex = gd.RipsComplex(points=ob_data)
    simplex_tree = rips_complex.create_simplex_tree(max_dimension = 2)
    diag = simplex_tree.persistence()
    A = simplex_tree.persistence_intervals_in_dimension(1)
    x1 = np.append(np.array([A.T[0]]),[A.T[1]-A.T[0]],axis=0)
    x = x1.T
    n = len(x)
    m = len(x[0])
    if len(x) > 0:
        KC = get_WKC(x, n, m, gamma, epsilon, param)
        KCs_PNPM.append(KC)
    else:
        KCs_PNPM.append(0)

We apply sequential MDL-change statistics (SMDL) to the series of the kernel complexity of PNPM.

In [ ]:
h = 70
cps_true = np.array([250,750])
N_trial = 1
mu_max = 50.0
sigma_min = 0.005
T = 200

scores_list_0th = []
scores_list_1st = []
scores_list_2nd = []
for i in range(N_trial):
    X = np.array(KCs_PNPM)
    len_X = len(X)
    
    norm1d = my_Norm1D()
    smdl = SMDL(norm1d)

    scores_0th = np.array([np.nan]*h + [ smdl.calc_change_score(X[(t-h):(t+h)], h, mu_max=mu_max, sigma_min=sigma_min) \
                                     for t in range(h, len_X-h)] + [np.nan]*h)
    scores_list_0th.append(scores_0th)

scores_list_0th = np.array(scores_list_0th)
auc_list_0th = calc_auc_average(scores_list_0th, cps_true=cps_true,T=T)
print("AUC:", np.mean(auc_list_0th))

## Comparison with existing methods
Below we apply several existing methods to the time-series for comparison.

### L2 norm of persistence landscape

In [ ]:
L2_norms = []
for i in range(len(use_data)):
    ob_data = use_data[i]
    rips_complex = gd.RipsComplex(points=ob_data)
    simplex_tree = rips_complex.create_simplex_tree(max_dimension=2)
    simplex_tree.persistence()
    A = simplex_tree.persistence_intervals_in_dimension(1)
    x1 = np.append(np.array([A.T[0]]),[A.T[1]-A.T[0]],axis=0)
    x = x1.T
    LS = Landscape(num_landscapes=3,resolution=1000)
    L = LS.fit_transform([simplex_tree.persistence_intervals_in_dimension(1)])
    L2 = 0
    L2 += pow(np.linalg.norm(L[0][:1000],ord=2),2)
    L2 += pow(np.linalg.norm(L[0][1000:2000],ord=2),2)
    L2 += pow(np.linalg.norm(L[0][2000:3000],ord=2),2)
    L2_norms.append(pow(L2,1/2))

In [ ]:
h = 20
cps_true = np.array([250,750])
N_trial = 1
mu_max = 50.0
sigma_min = 0.005
T = 200

scores_list_0th = []
scores_list_1st = []
scores_list_2nd = []
for i in range(N_trial):
    X = np.array(L2_norms)
    len_X = len(X)
    
    norm1d = Norm1D()
    smdl = SMDL(norm1d)

    scores_0th = np.array([np.nan]*h + [ smdl.calc_change_score(X[(t-h):(t+h)], h, mu_max=mu_max, sigma_min=sigma_min) \
                                     for t in range(h, len_X-h)] + [np.nan]*h)
    scores_list_0th.append(scores_0th)
    
scores_list_0th = np.array(scores_list_0th)
auc_list_0th = calc_auc_average(scores_list_0th, cps_true=cps_true,T=T)
print("AUC:", np.mean(auc_list_0th))

### Sequential MDL-change statistics (SMDL)

In [ ]:
h = 10
cps_true=np.array([500,1000])
N_trial = 1
mu_max = 50.0
sigma_min = 0.005
T = 200

scores_list_0th = []
scores_list_1st = []
scores_list_2nd = []
for i in range(N_trial):
    X = np.array(data)
    len_X = len(X)
    
    norm1d = Norm1D()
    smdl = SMDL(norm1d)

    scores_0th = np.array([np.nan]*h + [ smdl.calc_change_score(X[(t-h):(t+h)], h, mu_max=mu_max, sigma_min=sigma_min) \
                                     for t in range(h, len_X-h)] + [np.nan]*h)
    scores_list_0th.append(scores_0th)

scores_list_0th = np.array(scores_list_0th)
auc_list_0th = calc_auc_average(scores_list_0th, cps_true=cps_true,T=T)
print("AUC:", np.mean(auc_list_0th))

### Bayesian online change point detection (BOCPD)

In [ ]:
ALPHA = 0.1
BETA = 1.0
KAPPA = 1.0
MU = 0.0
DELAY = 15
T = 200

for LAMBDA in [100,600]:
    for THRESHOLD in [0.1, 0.3]:
        scores_bocpd = []
        for i in range(N_trial):
            X = data

            # BOCPD
            bocd = BOCD(partial(constant_hazard, LAMBDA),
                        StudentT(ALPHA, BETA, KAPPA, MU), X)
            change_points = []
            scores = [np.nan] * DELAY
            for x in X[:DELAY]:
                bocd.update(x)
            for x in X[DELAY:]:
                bocd.update(x)
                if bocd.growth_probs[DELAY] >= THRESHOLD:
                    change_points.append(bocd.t - DELAY + 1)
                score = np.sum(bocd.growth_probs[:bocd.t - DELAY] * 1.0 / (1.0 + np.arange(1, bocd.t - DELAY + 1)))
                scores.append(score)

            scores_bocpd.append(scores)

        scores_bocpd = np.array(scores_bocpd)
        auc_list = calc_auc_average(scores_bocpd,np.array([500,1000]),T=T)
        print('LAMBDA =', LAMBDA, 'THRESHOLD =', THRESHOLD, 'AUC:', np.mean(auc_list))